In [1]:
%pip install langchain==0.0.319
%pip install openai==0.28.1
%pip install chromadb==0.4.14
%pip install sentence-transformers
%pip install pypdf pdfplumber
%restart_python

  Using cached langchain-0.0.319-py3-none-any.whl.metadata (15 kB)
  Using cached langsmith-0.0.92-py3-none-any.whl.metadata (9.9 kB)
Using cached langchain-0.0.319-py3-none-any.whl (1.9 MB)
Using cached langsmith-0.0.92-py3-none-any.whl (56 kB)
  Attempting uninstall: langsmith
    Found existing installation: langsmith 0.1.147
    Uninstalling langsmith-0.1.147:
      Successfully uninstalled langsmith-0.1.147
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Attempting uninstall: langchain
    Found existing installation: langchain 0.1.16
    Uninstalling langchain-0.1.16:90m╺━━━━━━━━━━━━━━━━━━━ 1/2 [langchain]
      Successfully uninstalled langchain-0.1.16━━━━━━━━━━━━━━━━━━━ 1/2 [langchain]
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [langchain]  WARNING: The scripts langchain and langchain-server are installed in '/usr/local/python/3.12.1/bin' which is not on PATH.
  Consider adding this directory to PATH or,

UsageError: Line magic function `%restart_python` not found.


In [2]:
import os
import re
import json
import shutil
import hashlib
import logging
from typing import List, Dict, Optional

import pdfplumber
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma

# === Logging Setup ===
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

# === Configuration ===
pdf_files = [
    "Jockey202425.pdf",
    "HKJC_140A.pdf",
    "HKJC_Annual Report 2425 part1.pdf",
    "HKJC_Annual Report 2425 part2.pdf"
]
persist_dir = "./chroma_db_1223_8"
embedding_model_name = "sentence-transformers/all-MiniLM-L6-v2"

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    separators=["\n\n", "\n", ".", " ", ""]
)

seen_hashes = set()
all_chunks: List[Document] = []
table_count = 0


# === Helpers ===

def normalize_number(val: str) -> Optional[int]:
    try:
        val = val.replace(",", "").strip()
        if "(" in val and ")" in val:
            return -int(val.replace("(", "").replace(")", ""))
        return int(val)
    except Exception:
        return None


def extract_section_title(lines: List[str]) -> str:
    for line in lines:
        if len(line.strip()) > 5 and line.isupper():  # crude heuristic
            return line.strip()
    return "Unknown Section"


def extract_cashflow_rows(lines: List[str], pdf_path: str, page_num: int) -> List[Document]:
    global table_count
    docs = []
    header_years = []
    header_line_index = None

    # Find header row
    for idx, line in enumerate(lines):
        found_years = re.findall(r"\b(20\d{2})\b", line)
        if len(found_years) >= 2:
            header_years = found_years
            header_line_index = idx
            break

    if not header_years:
        return []

    section_title = extract_section_title(lines[:header_line_index])

    for line in lines[header_line_index + 1:]:
        if len(line.strip()) < 10:
            continue

        numbers = re.findall(r"-?\(?\d[\d,]*\)?", line)
        if len(numbers) < len(header_years):
            continue

        label_match = re.match(r"^(.*?)\s{2,}", line)
        label = label_match.group(1).strip() if label_match else line.strip()

        value_dict = {}
        for idx, year in enumerate(header_years):
            if idx < len(numbers):
                val = normalize_number(numbers[idx])
                if val is not None:
                    value_dict[year] = val

        if not value_dict:
            continue

        summary = "\n".join([f"{year}: {val} million HKD" for year, val in value_dict.items()])
        table_count += 1

        doc = Document(
            page_content=f"{label}:\n{summary}",
            metadata={
                "source": pdf_path,
                "type": "cashflow_row",
                "table_id": f"{pdf_path}_row_{table_count}",
                "label": label,
                "section": section_title,
                "page_start": page_num,
                "page_end": page_num,
                "columns": ", ".join(header_years),
                "json": json.dumps({"label": label, **value_dict}),
                "fallback_prompt": f"Row from {section_title}: {label}"
            }
        )

        hash_ = hashlib.md5(doc.page_content.encode()).hexdigest()
        if hash_ not in seen_hashes:
            docs.append(doc)
            seen_hashes.add(hash_)

    return docs


def extract_tables_from_pdfplumber(page, pdf_path: str, page_num: int) -> List[Document]:
    global table_count
    docs = []

    try:
        tables = page.extract_tables()
        for table in tables:
            if not table or len(table) < 2:
                continue

            headers = table[0]
            # Normalize headers
            headers = [h.strip().lower() if h else "" for h in headers]

            # Identify year columns (e.g., "2025", "2024")
            year_columns = {}
            label_col = 0
            for idx, header in enumerate(headers):
                if re.fullmatch(r"\d{4}", header):
                    year_columns[header] = idx

            if not year_columns:
                continue

            for row in table[1:]:
                if len(row) < max(year_columns.values()) + 1:
                    continue

                label = row[label_col].strip() if label_col < len(row) and row[label_col] else "Unknown"
                if not label or len(label) < 3:
                    continue

                value_dict = {}
                for year, idx in year_columns.items():
                    if idx >= len(row):
                        continue
                    val = normalize_number(row[idx])
                    if val is not None:
                        value_dict[year] = val

                if not value_dict:
                    continue

                summary = "\n".join([f"{year}: {val} million HKD" for year, val in value_dict.items()])
                table_count += 1

                doc = Document(
                    page_content=f"{label}:\n{summary}",
                    metadata={
                        "source": pdf_path,
                        "type": "table_row",
                        "table_id": f"{pdf_path}_table_{table_count}",
                        "label": label,
                        "page_start": page_num,
                        "page_end": page_num,
                        "columns": ", ".join(year_columns.keys()),
                        "json": json.dumps({"label": label, **value_dict}),
                        "fallback_prompt": f"Table row from page {page_num}: {label}"
                    }
                )

                hash_ = hashlib.md5(doc.page_content.encode()).hexdigest()
                if hash_ not in seen_hashes:
                    docs.append(doc)
                    seen_hashes.add(hash_)

    except Exception as e:
        logging.warning(f"❌ Table extraction failed on {pdf_path} page {page_num}: {e}")

    return docs


def evaluate_chunk_quality(doc: Document) -> bool:
    """Very simple heuristic evaluation to skip low-content chunks."""
    content = doc.page_content.strip()
    if len(content) < 100:
        return False
    if content.lower().count("page") > 5:
        return False
    return True


# === Main Processing ===

for pdf_path in pdf_files:
    logging.info(f"📄 Processing: {pdf_path}")

    try:
        # Load with PyPDFLoader
        loader = PyPDFLoader(pdf_path)
        docs = loader.load()

        for doc in docs:
            doc.metadata.update({
                "source": pdf_path,
                "type": "text",
                "fallback_prompt": "You are a PDF assistant. If data is unclear or missing, say 'Not enough information.'",
            })

        # Split and deduplicate
        text_chunks = text_splitter.split_documents(docs)
        for chunk in text_chunks:
            if not evaluate_chunk_quality(chunk):
                continue
            content_hash = hashlib.md5(chunk.page_content.strip().encode()).hexdigest()
            if content_hash not in seen_hashes:
                all_chunks.append(chunk)
                seen_hashes.add(content_hash)

        # Table and cashflow extraction
        with pdfplumber.open(pdf_path) as pdf:
            for i, page in enumerate(pdf.pages):
                text = page.extract_text()
                lines = text.splitlines() if text else []

                cash_docs = extract_cashflow_rows(lines, pdf_path, i + 1)
                all_chunks.extend(cash_docs)

                table_docs = extract_tables_from_pdfplumber(page, pdf_path, i + 1)
                all_chunks.extend(table_docs)

                # Fallback plain text page
                if not cash_docs and not table_docs and text:
                    fallback_doc = Document(
                        page_content=text,
                        metadata={
                            "source": pdf_path,
                            "type": "page_text",
                            "page": i + 1,
                            "fallback_prompt": "This page may contain financial or contextual information."
                        }
                    )
                    hash_ = hashlib.md5(fallback_doc.page_content.encode()).hexdigest()
                    if hash_ not in seen_hashes:
                        all_chunks.append(fallback_doc)
                        seen_hashes.add(hash_)

    except Exception as e:
        logging.error(f"❌ Failed to process {pdf_path}: {e}")

# === Embedding and Persisting ===

if os.path.exists(persist_dir):
    shutil.rmtree(persist_dir)

if not all_chunks:
    logging.error("❌ No chunks to embed.")
else:
    logging.info("📦 Creating embeddings and saving to vectorstore...")

    embedding = HuggingFaceEmbeddings(
        model_name=embedding_model_name,
        model_kwargs={"device": "cpu"},
        encode_kwargs={"normalize_embeddings": True}
    )

    vectordb = Chroma.from_documents(
        documents=all_chunks,
        embedding=embedding,
        persist_directory=persist_dir
    )
    vectordb.persist()

    logging.info("✅ Embeddings created and saved successfully!")
    logging.info(f"📁 Vectorstore directory: {persist_dir}")
    logging.info(f"📄 Total Chunks Embedded: {len(all_chunks)}")
    logging.info(f"📊 Cashflow Rows Extracted: {len([d for d in all_chunks if d.metadata.get('type') == 'cashflow_row'])}")
    logging.info(f"📋 Table Rows Extracted: {len([d for d in all_chunks if d.metadata.get('type') == 'table_row'])}")
    logging.info(f"📃 Fallback Text Chunks: {len([d for d in all_chunks if d.metadata.get('type') == 'page_text'])}")

2025-12-23 14:55:49,224 - INFO - 📄 Processing: Jockey202425.pdf


2025-12-23 14:55:49,650 - INFO - 📄 Processing: HKJC_140A.pdf
2025-12-23 14:56:18,224 - INFO - 📄 Processing: HKJC_Annual Report 2425 part1.pdf
2025-12-23 14:56:46,425 - INFO - 📄 Processing: HKJC_Annual Report 2425 part2.pdf
2025-12-23 14:57:02,046 - INFO - 📦 Creating embeddings and saving to vectorstore...
2025-12-23 14:57:02,047 - INFO - Load pretrained SentenceTransformer: sentence-transformers/all-MiniLM-L6-v2
2025-12-23 14:57:05,986 - INFO - Anonymized telemetry enabled. See https://docs.trychroma.com/telemetry for more information.
2025-12-23 14:57:06,057 - ERROR - Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
2025-12-23 14:57:06,068 - ERROR - Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given
Batches: 100%|██████████| 67/67 [01:53<00:00,  1.70s/it]
2025-12-23 14:59:02,762 - INFO - ✅ Embeddings created and saved successfully!
2025-12-23 14:59:02,763 - INFO - 📁 V

In [ ]:
%pip install --upgrade langchain==0.1.16 langchain-community==0.0.33 langchain-openai==0.0.8


In [ ]:
import os
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("HKJC_140A.pdf")
docs = loader.load()
print(f"📄 Loaded {len(docs)} pages")
print("📝 Sample text:", docs[0].page_content[:300])
# ✅ Vectorstore path (same as you used)
persist_dir = "./chroma_db_1222_5"

# ✅ Load the same embedding model used to build the DB
embedding = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

# ✅ Load the vectorstore from disk
vectordb = Chroma(persist_directory=persist_dir, embedding_function=embedding)

# ✅ Check how many documents are in the store
print("📄 Number of documents in vectorstore:", vectordb._collection.count())

# ✅ Ask a test question
user_query = "who are An An and Ke Ke?"
results = vectordb.similarity_search(user_query, k=3)

# ✅ Print results
if results:
    for i, doc in enumerate(results, 1):
        print(f"\n🔹 Result {i}:\n{doc.page_content[:500]}")  # truncate output
else:
    print("❌ No relevant documents found.")

for i, doc in enumerate(docs[:3]):
    print(f"\nPage {i+1} content length: {len(doc.page_content)}")
    print(doc.page_content[:300])

📄 Loaded 118 pages
📝 Sample text: 


Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event CollectionQueryEvent: capture() takes 1 positional argument but 3 were given


📄 Number of documents in vectorstore: 1001

🔹 Result 1:
3
Board of Stewards
董事局
Mr Michael T H Lee 
JP 
(Chairman)
利子厚先生(主席)
The Hon Martin Liao 
GBM GBS JP 
(Deputy Chairman)
廖長江先生(副主席)
Dr Silas S S Yang 
JP
楊紹信博士
Miss Anita Fung Yuen Mei 
BBS JP
馮婉眉小姐
Mr Philip K W Lo
羅啟華先生
Mr Andrew W B R Weir 
BBS MBE JP
韋安祖先生
Mr Lester G Huang 
SBS JP
黃嘉純先生
Dr Henry H L Chan
陳衍里醫生
The Hon Bernard Charnwut Chan 
GBM GBS JP
陳智思先生
Mr Jackson Woo Ka Biu
胡家驃先生
Mrs Ann Kung Yeung Yun Chi 
BBS JP
龔楊恩慈女士
Mr Nicholas D Hunsworth
孔思和先生

🔹 Result 2:

SECTOR

🔹 Result 3:
	BEAUTY JOY
美麗同享
A S Cruz
告東尼
Eleanor Kwok Law Kwai
Chun & Patrick Kwok Ho
Chuen
郭羅桂珍與郭浩泉	

Page 1 content length: 0


Page 2 content length: 0


Page 3 content length: 0

